<a href="https://colab.research.google.com/github/patriciamedyna/Classifier_page/blob/main/Bert_victor_tags.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Datasets/Victor/small_victor')
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

Mounted at /content/drive/


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.7 MB/s eta 0:00:00


In [3]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.6 MB/s eta 0:00:00


In [4]:
import torch

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [5]:
def process_data_for_bert(data):

    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    X_tokenized = tokenizer(list(data['text']), padding=True, truncation=True, max_length=512)
    y = list(data['label'])

    dataset = Dataset(X_tokenized, y)

    return dataset

In [6]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoModel, AutoTokenizer
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import AutoModelForSequenceClassification
from transformers import AutoModelForMaskedLM
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

def runBert(df_train_dataset, df_val_dataset, df_test_dataset):

    batch_size=16

    model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=False)
    #model = AutoModelForPreTraining.from_pretrained('neuralmind/bert-base-portuguese-cased')
    #tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)

    X_train, X_val, y_train, y_val = train_test_split(df_train_dataset['text'], df_train_dataset['label'],
                                                    test_size=0.2,
                                                    random_state=42)

    train_dataset = pd.DataFrame({'text':X_train, 'label':y_train})
    val_dataset = pd.DataFrame({'text':X_val, 'label':y_val})

    train_dataset = process_data_for_bert(df_train_dataset)
    val_dataset = process_data_for_bert(df_val_dataset)
    test_dataset = process_data_for_bert(df_test_dataset)

    # ----- 2. Fine-tune pretrained model -----#
    # Define Trainer parameters
    def compute_metrics(p):

        pred, labels = p
        pred = np.argmax(pred, axis=1)

        accuracy = accuracy_score(y_true=labels, y_pred=pred)
        recall = recall_score(y_true=labels, y_pred=pred, average='micro')
        precision = precision_score(y_true=labels, y_pred=pred, average='micro')
        f1 = f1_score(y_true=labels, y_pred=pred, average='micro')

        return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

    # Define Trainer
    args = TrainingArguments(
        "./model/layoutqt_victor2",
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=10,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='f1'
      # optim="adamw_torch"
    )

    def model_init():
        return model

    trainer = Trainer(
        model_init=model_init,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # New code - wrap collator in a dictionary
    #data_collator = trainer.data_collator
    #trainer.data_collator = lambda data: dict(data_collator(data))
    # End new code

    trainer.train()

    # Make prediction
    raw_pred, _, _ = trainer.predict(test_dataset)

    # Preprocess raw predictions
    y_pred = np.argmax(raw_pred, axis=1)

    return y_pred
    #return roc_auc_score(df_test_dataset['class'], y_pred), f1_score(df_test_dataset['class'], y_pred, average='macro')

In [7]:
import numpy as np
import pandas as pd


df_train = pd.read_csv("./train/df_train_tags_first.csv")
#df_train = df_train[0:10000]

df_val = pd.read_csv("./val/df_val_tags_first.csv")
#df_val = df_val[0:2000]

df_test = pd.read_csv("./test/df_test_tags_first.csv")
#df_test = df_test[0:2000]

df_train['text'] = df_train['text'].apply(lambda x: str(x))
df_test['text'] = df_test['text'].apply(lambda x: str(x))

#df_train['label'] = df_train['label'].apply(lambda x: int(x))
#df_test['label'] = df_test['label'].apply(lambda x: int(x))



In [8]:
# Dicionário de mapeamento
mapeamento = {'acordao_de_2_instancia': 0, 'agravo_em_recurso_extraordinario': 1, 'despacho_de_admissibilidade': 2, 'outros': 3, 'peticao_do_RE': 4, 'sentenca': 5}

# Função de conversão
def converter_rotulo(rotulo):
 return mapeamento[rotulo]

# Aplicar a conversão na coluna 'label'
df_train['label'] = df_train['label'].astype(str).apply(converter_rotulo)
df_val['label'] = df_val['label'].astype(str).apply(converter_rotulo)
df_test['label'] = df_test['label'].astype(str).apply(converter_rotulo)

#print(df_val)

In [ ]:
#df_test['label']

In [9]:
y_pred = runBert(df_train, df_val, df_test)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.123200,0.118000,0.971124,0.971124,0.971124,0.971124
2,0.090900,0.085054,0.977607,0.977607,0.977607,0.977607
3,0.075200,0.086155,0.975957,0.975957,0.975957,0.975957
4,0.055700,0.088871,0.978353,0.978353,0.978353,0.978353
5,0.045500,0.098784,0.975171,0.975171,0.975171,0.975171
6,0.037200,0.099950,0.979885,0.979885,0.979885,0.979885
7,0.032800,0.100254,0.978392,0.978392,0.978392,0.978392
8,0.024300,0.117982,0.980278,0.980278,0.980278,0.980278
9,0.017400,0.114037,0.978510,0.978510,0.978510,0.978510
10,0.016200,0.120930,0.979650,0.979650,0.979650,0.979650


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score

print("Accuracy:", accuracy_score(df_test['label'], y_pred))
print("F1:", f1_score(df_test['label'], y_pred, average='micro'))

Accuracy: 0.9785787684768003
F1: 0.9785787684768003


In [ ]:
print(classification_report(df_test['label'], y_pred))

              precision    recall  f1-score   support

           0       0.92      0.93      0.92       197
           1       0.65      0.50      0.57       203
           2       0.83      0.51      0.64       146
           3       0.99      0.99      0.99     24193
           4       0.76      0.68      0.72       301
           5       0.89      0.82      0.85       262

    accuracy                           0.98     25302
   macro avg       0.84      0.74      0.78     25302
weighted avg       0.98      0.98      0.98     25302



In [ ]:
print(classification_report(df_test['label'], y_pred))

              precision    recall  f1-score   support

           0       0.96      0.89      0.93       197
           1       0.65      0.49      0.56       203
           2       0.84      0.36      0.50       146
           3       0.98      0.99      0.99     24193
           4       0.77      0.65      0.71       301
           5       0.84      0.77      0.80       262

    accuracy                           0.98     25302
   macro avg       0.84      0.69      0.75     25302
weighted avg       0.98      0.98      0.98     25302



In [10]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score

print("Accuracy:", accuracy_score(df_test['label'], y_pred))
print("F1:", f1_score(df_test['label'], y_pred, average='micro'))

Accuracy: 0.9812663030590467
F1: 0.9812663030590467


In [11]:
print(classification_report(df_test['label'], y_pred))

              precision    recall  f1-score   support

           0       0.93      0.92      0.93       197
           1       0.65      0.59      0.62       203
           2       0.73      0.64      0.68       146
           3       0.99      0.99      0.99     24193
           4       0.77      0.69      0.73       301
           5       0.89      0.84      0.86       262

    accuracy                           0.98     25302
   macro avg       0.83      0.78      0.80     25302
weighted avg       0.98      0.98      0.98     25302

